In [1]:
import os
import gc
import numpy as np
import pandas as pd
import math
import random
from time import time
from tqdm import tqdm
from pathlib import Path

import torch
from torch import nn, cuda
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset

from torch.optim import Adam, SGD, Optimizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from utils import CosineAnnealingWithRestartsLR

import warnings
warnings.filterwarnings('ignore')

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class Semi_dataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.X_dataset = []
        self.Y_dataset = []
        for x in X:
            self.X_dataset.append(torch.FloatTensor(x))
        try:
            for y in Y.values:
                self.Y_dataset.append(torch.tensor(y))
        except:
            print("no label")
            
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        data = self.X_dataset[index]
        try:
            target = self.Y_dataset[index]
            return data, target
        except:
            return data


def build_dataloader(X, Y, batch_size, shuffle=False):
    
    dataset = Semi_dataset(X, Y)
    dataloader = DataLoader(
                            dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            num_workers=0
                            )
    return dataloader

def mean_absolute_error(y_true, y_pred,
                        sample_weight=None,
                        multioutput='uniform_average'):
    
    output_errors = np.average(np.abs(y_pred - y_true),
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            multioutput = None

    return np.average(output_errors, weights=multioutput)

class AdamW(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay)
        super(AdamW, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('AdamW does not support sparse gradients, please consider SparseAdam instead')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(-step_size, exp_avg, denom)

                if group['weight_decay'] != 0:
                    p.data.add_(-group['weight_decay'], p.data)

        return loss

class MLP_only_flatfeatures(nn.Module):
    def __init__(self, num_classes=1):
        super(MLP_only_flatfeatures, self).__init__()
        self.num_classes = num_classes         
        self.fc_layers = nn.Sequential(
            #nn.Linear(226, 1000),
            nn.Linear(230, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            ####### Block 1 #######
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            ####### Block 2 #######
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            ####### Block 3 #######
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            #######################
            
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            
            nn.Dropout(0.1),
            nn.Linear(512, self.num_classes)
            )             
        self._initialize_weights()

    def forward(self, x):
        out = self.fc_layers(x)
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def build_model(device, model_name='mlp', weight_path=None):

    if model_name == 'mlp':
        model = MLP_only_flatfeatures(4)
    
    return model

def validation(model, criterion, valid_loader, device):
    
    model.eval()
    valid_preds = np.zeros((len(valid_loader.dataset), 4))
    valid_targets = np.zeros((len(valid_loader.dataset), 4))
    val_loss = 0.
    
    with torch.no_grad():
        for i, (data, target) in enumerate(valid_loader):
            
            valid_targets[i * batch_size: (i+1) * batch_size] = target.float().numpy().copy()

            data = data.to(device)
            target = target.float().to(device)
                
            output = model(data)
            loss = criterion(output, target)
            
            valid_preds[i * batch_size: (i+1) * batch_size] = output.detach().cpu().numpy()
            
            val_loss += loss.item() / len(valid_loader)
    
    val_score = mean_absolute_error(valid_preds, valid_targets)

    return val_loss, val_score

seed = 42
seed_everything(seed)

if cuda.is_available:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [3]:
%%time
DATASET_PATH = '../wafer'
train_df = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATASET_PATH, 'test.csv'))
submission_df = pd.read_csv(os.path.join(DATASET_PATH, 'sample_submission.csv'))

CPU times: user 13.2 s, sys: 1.89 s, total: 15 s
Wall time: 12.2 s


In [4]:
%%time
train_df['226'] = train_df.iloc[:,4:229].std(axis=1)
train_df['227'] = train_df.iloc[:,4:229].min(axis=1)
train_df['228'] = train_df.iloc[:,4:229].max(axis=1)
train_df['229'] = train_df.iloc[:,4:229].mean(axis=1)

test_df['226'] = test_df.iloc[:,1:226].std(axis=1)
test_df['227'] = test_df.iloc[:,1:226].min(axis=1)
test_df['228'] = test_df.iloc[:,1:226].max(axis=1)
test_df['229'] = test_df.iloc[:,1:226].mean(axis=1)

CPU times: user 12.5 s, sys: 1.18 s, total: 13.7 s
Wall time: 4.05 s


In [5]:
X_train, X_val, y_train, y_val = train_test_split(train_df.iloc[:, 4:], train_df.iloc[:, :4], test_size=0.1, random_state=42, shuffle=True)

X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

batch_size = 2048
train_loader = build_dataloader(X_train, y_train, batch_size, shuffle=True)
valid_loader = build_dataloader(X_val, y_val, batch_size, shuffle=False)

test_df.iloc[:, 1:] = scaler.transform(test_df.iloc[:, 1:])

In [6]:
# output path
#output_dir = Path('./', 'output')
#output_dir.mkdir(exist_ok=True, parents=True)

num_epochs = 1000
criterion = nn.L1Loss()

best_epoch_list = []
best_valid_score_list = []

# build model
model = build_model(device, model_name='mlp')
model.to(device)

lr = 0.001
optimizer = AdamW(model.parameters(), lr)

eta_min = 0.000001
T_max = 10
T_mult = 1
restart_decay = 0.97
scheduler = CosineAnnealingWithRestartsLR(optimizer, T_max=T_max, eta_min=eta_min, T_mult=T_mult, restart_decay=restart_decay)


start_time = time()

best_epoch = 0
best_train_loss = 1000
best_valid_score = 1000

for epoch in range(num_epochs):

    model.train()
    optimizer.zero_grad()
    train_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):

        if device:
            data = data.to(device)
            target = target.float().to(device)
        else:
            target = target.float()

        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item() / len(train_loader)

    val_loss, val_score = validation(model, criterion, valid_loader, device)

    elapsed = time() - start_time

    lr = [_['lr'] for _ in optimizer.param_groups]

    scheduler.step(val_score)
    
    print('Epoch {} / {}  train Loss: {:.4f}  val_loss: {:.4f}  val_score: {:.4f}  lr: {:.5f}  elapsed: {:.0f}m {:.0f}s' \
          .format(epoch,  num_epochs - 1, train_loss, val_loss, val_score, lr[0], elapsed // 60, elapsed % 60))
        
    #model_path = output_dir / 'best_model.pt'
    model_path = '../wafer/mlp_weights/mlp_v1_stats_1000e_cosine.pt'

    if val_score < best_valid_score:
        best_valid_score = val_score
        best_epoch = epoch
        torch.save(model.state_dict(), model_path)
        print('----------------------------------------------------------------------->> loss improved to {:.5f}'.format(best_valid_score))

    best_epoch_list.append(best_epoch)
    best_valid_score_list.append(best_valid_score)
print("==================== mlp - Best val_loss - {:.5f} =================".format(best_valid_score))

d: 77m 41s
Epoch 812 / 999  train Loss: 3.1970  val_loss: 0.9082  val_score: 0.9081  lr: 0.00080  elapsed: 77m 47s
Epoch 813 / 999  train Loss: 3.1980  val_loss: 0.9337  val_score: 0.9337  lr: 0.00079  elapsed: 77m 53s
Epoch 814 / 999  train Loss: 3.1954  val_loss: 0.9548  val_score: 0.9545  lr: 0.00080  elapsed: 77m 58s
Epoch 815 / 999  train Loss: 3.1972  val_loss: 0.9413  val_score: 0.9412  lr: 0.00080  elapsed: 78m 4s
Epoch 816 / 999  train Loss: 3.1984  val_loss: 0.9593  val_score: 0.9592  lr: 0.00080  elapsed: 78m 10s
Epoch 817 / 999  train Loss: 3.2056  val_loss: 0.9130  val_score: 0.9135  lr: 0.00080  elapsed: 78m 16s
Epoch 818 / 999  train Loss: 3.1993  val_loss: 0.9489  val_score: 0.9489  lr: 0.00080  elapsed: 78m 21s
Epoch 819 / 999  train Loss: 3.2035  val_loss: 0.9484  val_score: 0.9482  lr: 0.00080  elapsed: 78m 27s
Epoch 820 / 999  train Loss: 3.1941  val_loss: 0.9527  val_score: 0.9528  lr: 0.00080  elapsed: 78m 33s
Epoch 821 / 999  train Loss: 3.1928  val_loss: 0.8982 

In [7]:
epoch_df = pd.DataFrame()
epoch_df['epoch'] = best_epoch_list
epoch_df['val_score'] = best_valid_score_list

In [8]:
epoch_df.sort_values('val_score').head()

,epoch,val_score
999,953,0.830861
972,953,0.830861
971,953,0.830861
970,953,0.830861
969,953,0.830861


In [9]:
score_to = round(min(best_valid_score_list),6)
score_to

0.830861

In [11]:
%%time
batch_size = 2048
test_loader = build_dataloader(test_df.iloc[:, 1:].values, Y=None, batch_size=batch_size, shuffle=False)

model = build_model(device, model_name='mlp')
model.to(device)

model.eval()
model.load_state_dict(torch.load(model_path))

test_preds = np.zeros((len(test_loader.dataset), 4))

with torch.no_grad():
    for batch_idx, data in enumerate(test_loader):
        if device:
            data = data.to(device)
        outputs = model(data)
        test_preds[batch_idx * batch_size:(batch_idx+1) * batch_size] = outputs.detach().cpu().numpy()

no label
CPU times: user 86.2 ms, sys: 16 ms, total: 102 ms
Wall time: 103 ms


In [12]:
submission = pd.DataFrame({'id': submission_df['id'],
                           'layer_1':test_preds.transpose()[0],
                           'layer_2':test_preds.transpose()[1],
                           'layer_3':test_preds.transpose()[2],
                           'layer_4':test_preds.transpose()[3]})
submission.to_csv('../wafer/mlp_submission/mlp_v1_stats_1000e_cosine_{}_submission.csv'.format(score_to), index=False)

submission.head()

,id,layer_1,layer_2,layer_3,layer_4
0,0,253.882034,230.181641,130.377167,89.371307
1,1,158.991135,127.112167,235.546143,99.952805
2,2,148.341812,176.705551,273.988434,155.042053
3,3,91.433372,230.720779,187.542053,82.531372
4,4,275.130920,295.080292,243.270081,272.185272
